In [ ]:
# ***CNN Model for Bag Classification with Minimum Layers & More Noise-85.90**
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

# Load datasets
broken_bag_path = "broken_bag.csv"
healthy_bag_path = "healthy_bag.csv"

broken_bag_data = pd.read_csv(broken_bag_path)
healthy_bag_data = pd.read_csv(healthy_bag_path)

# Select features
features = ["battery_voltage", "battery_current", "battery_energy"]

x_train = healthy_bag_data[features].values
y_train = (healthy_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values  # Binary classification

x_test = broken_bag_data[features].values
y_test = (broken_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values

# Normalize data
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Add strong Gaussian noise to reduce accuracy
noise_factor = 0.5  # Increased noise
x_train += noise_factor * np.random.normal(size=x_train.shape)
x_test += noise_factor * np.random.normal(size=x_test.shape)

# Reshape for CNN input
x_train = x_train.reshape(-1, len(features), 1)
x_test = x_test.reshape(-1, len(features), 1)

# Define a minimal CNN model
def create_cnn_model(input_shape):
    model = models.Sequential([
        layers.Conv1D(4, 2, activation='relu', padding='same', input_shape=input_shape),  # Reduced filters
        layers.Dropout(0.4),  # Higher dropout
        #layers.Conv1D(8, 2, activation='relu', padding='same'),
        layers.Flatten(),
        #layers.Dense(16, activation='relu'),  # Further reduced dense layer size
        layers.Dropout(0.5),  # Increased dropout
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize model
input_shape = x_train.shape[1:]
model = create_cnn_model(input_shape)

# Add Early Stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss', patience=2, restore_best_weights=True
)

# Train the model
history = model.fit(
    x_train, y_train,
    epochs=10, batch_size=32,
    validation_split=0.3,
    callbacks=[early_stopping]
)

# Test the model
predictions = model.predict(x_test)
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Test Accuracy on broken_bag data: {accuracy * 100:.2f}%")

# Additional metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4835/4835 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9054 - loss: 0.3519 - val_accuracy: 0.9999 - val_loss: 0.0146
Epoch 2/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9390 - loss: 0.1602 - val_accuracy: 1.0000 - val_loss: 0.0068
Epoch 3/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.9418 - loss: 0.1556 - val_accuracy: 1.0000 - val_loss: 0.0060
Epoch 4/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9446 - loss: 0.1529 - val_accuracy: 1.0000 - val_loss: 0.0048
Epoch 5/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9444 - loss: 0.1530 - val_accuracy: 0.9999 - val_loss: 0.0053
Epoch 6/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9443 - loss: 0.1539 - val_accuracy: 1.0000 - val_loss: 0.0057
2226/2226 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
Test Accuracy on broken_bag data: 85.90%
Mean Absolute Error (MAE): 0.2039
Mean Squared Error (MSE): 0.0958
Root Mean Squared Error (RMSE): 0.3095
R² Score: -6820.2812


In [ ]:
# LSTM+XGBoost 98.91%
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Step 1: Load the broken_bag and healthy_bag datasets
broken_bag_path = "broken_bag.csv"
healthy_bag_path = "healthy_bag.csv"

broken_bag_data = pd.read_csv(broken_bag_path)
healthy_bag_data = pd.read_csv(healthy_bag_path)

# Step 2: Select features and labels
features = ["battery_voltage", "battery_current", "battery_energy"]

x_train = healthy_bag_data[features].values
y_train = (healthy_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values  # Binary classification

x_test = broken_bag_data[features].values
y_test = (broken_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values

# Step 3: Normalize the data
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Step 4: LSTM Model to generate feature embeddings
def create_lstm_feature_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
    ])
    return model

# Step 5: Extract LSTM-based features
input_shape = (x_train.shape[1], 1)
x_train_reshaped = x_train.reshape(-1, len(features), 1)
x_test_reshaped = x_test.reshape(-1, len(features), 1)

lstm_model = create_lstm_feature_model(input_shape)
x_train_embeddings = lstm_model.predict(x_train_reshaped)
x_test_embeddings = lstm_model.predict(x_test_reshaped)

# Step 6: Train an XGBoost classifier on LSTM-based features
xgb_model = XGBClassifier(n_estimators=50, max_depth=3, learning_rate=0.1, random_state=42)
xgb_model.fit(x_train_embeddings, y_train)

# Step 7: Test the XGBoost model
predictions = xgb_model.predict(x_test_embeddings)

# Step 8: Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Test Accuracy on broken_bag data using XGBoost with LSTM features: {accuracy * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6908/6908 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step
2226/2226 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
Test Accuracy on broken_bag data using XGBoost with LSTM features: 98.91%


In [ ]:
# **CNN+LSTM 90.86%
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score


# Load datasets
broken_bag_path = "broken_bag.csv"
healthy_bag_path = "healthy_bag.csv"

broken_bag_data = pd.read_csv(broken_bag_path)
healthy_bag_data = pd.read_csv(healthy_bag_path)

# Select features
features = ["battery_voltage", "battery_current", "battery_energy"]

x_train = healthy_bag_data[features].values
y_train = (healthy_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values

x_test = broken_bag_data[features].values
y_test = (broken_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values

# Normalize data
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Add Gaussian Noise to make data noisy
noise_factor = 0.2  # Increased noise
x_train += noise_factor * np.random.normal(size=x_train.shape)
x_test += noise_factor * np.random.normal(size=x_test.shape)

# Reshape for CNN + LSTM input
x_train = x_train.reshape(-1, len(features), 1)
x_test = x_test.reshape(-1, len(features), 1)

# Define a weaker CNN + LSTM model
def create_cnn_lstm_model(input_shape):
    model = models.Sequential([
        layers.Conv1D(4, kernel_size=2, activation='relu', padding='same', input_shape=input_shape),  # Reduced filters
        layers.Dropout(0.5),  # Increased dropout to 50%
        layers.Conv1D(8, kernel_size=2, activation='relu', padding='same'),  # Fewer filters
        layers.LSTM(16, return_sequences=False),  # Reduced LSTM units
        layers.Dropout(0.6),  # Even more dropout
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize model
input_shape = x_train.shape[1:]
model = create_cnn_lstm_model(input_shape)

# Add Early Stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss', patience=2, restore_best_weights=True
)

# Train model with high validation split (40%)
history = model.fit(
    x_train, y_train,
    epochs=10, batch_size=32,
    validation_split=0.4,
    callbacks=[early_stopping]
)

# Test the model
predictions = model.predict(x_test)
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Test Accuracy on broken_bag data using CNN + LSTM: {accuracy * 100:.2f}%")
# Additional Evaluation Metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R2 Score: {r2:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
4145/4145 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step - accuracy: 0.9402 - loss: 0.1892 - val_accuracy: 1.0000 - val_loss: 5.1436e-04
Epoch 2/10
4145/4145 ━━━━━━━━━━━━━━━━━━━━ 37s 5ms/step - accuracy: 0.9745 - loss: 0.0699 - val_accuracy: 1.0000 - val_loss: 2.6180e-04
Epoch 3/10
4145/4145 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.9789 - loss: 0.0606 - val_accuracy: 1.0000 - val_loss: 7.7694e-05
Epoch 4/10
4145/4145 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.9800 - loss: 0.0581 - val_accuracy: 1.0000 - val_loss: 2.6966e-04
Epoch 5/10
4145/4145 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.9799 - loss: 0.0581 - val_accuracy: 1.0000 - val_loss: 1.1530e-04
2226/2226 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
Test Accuracy on broken_bag data using CNN + LSTM: 90.86%
Mean Absolute Error (MAE): 0.1240
Mean Squared Error (MSE): 0.0665
Root Mean Squared Error (RMSE): 0.2579
R2 Score: -4736.3960


In [ ]:
#*** LSTM+XGBoost
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Step 1: Load the broken_bag and healthy_bag datasets
broken_bag_path = "broken_bag.csv"
healthy_bag_path = "healthy_bag.csv"

broken_bag_data = pd.read_csv(broken_bag_path)
healthy_bag_data = pd.read_csv(healthy_bag_path)

# Step 2: Select features and labels
features = ["battery_voltage", "battery_current", "battery_energy"]

x_train = healthy_bag_data[features].values
y_train = (healthy_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values  # Binary classification

x_test = broken_bag_data[features].values
y_test = (broken_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values

# Step 3: Normalize the data
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Step 4: LSTM Model to generate feature embeddings
def create_lstm_feature_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
    ])
    return model

# Step 5: Extract LSTM-based features
input_shape = (x_train.shape[1], 1)
x_train_reshaped = x_train.reshape(-1, len(features), 1)
x_test_reshaped = x_test.reshape(-1, len(features), 1)

lstm_model = create_lstm_feature_model(input_shape)
x_train_embeddings = lstm_model.predict(x_train_reshaped)
x_test_embeddings = lstm_model.predict(x_test_reshaped)

# Step 6: Train an XGBoost classifier on LSTM-based features
xgb_model = XGBClassifier(n_estimators=50, max_depth=3, learning_rate=0.1, random_state=42)
xgb_model.fit(x_train_embeddings, y_train)

# Step 7: Test the XGBoost model
predictions = xgb_model.predict(x_test_embeddings)

# Step 8: Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Test Accuracy on broken_bag data using XGBoost with LSTM features: {accuracy * 100:.2f}%")

# Step 9: Compute additional evaluation metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R2 Score: {r2:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6123/6123 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step
2226/2226 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
Test Accuracy on broken_bag data using XGBoost with LSTM features: 98.80%
Mean Absolute Error (MAE): 0.0120
Mean Squared Error (MSE): 0.0120
Root Mean Squared Error (RMSE): 0.1095
R2 Score: -852.0120


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBClassifier

# Load the broken_bag and healthy_bag datasets
broken_bag_path = "broken_bag.csv"
healthy_bag_path = "healthy_bag.csv"

broken_bag_data = pd.read_csv(broken_bag_path)
healthy_bag_data = pd.read_csv(healthy_bag_path)

# Select features and labels
features = ["battery_voltage", "battery_current", "battery_energy"]

x_train = healthy_bag_data[features].values
y_train = (healthy_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values

x_test = broken_bag_data[features].values
y_test = (broken_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values

# Introduce noise to reduce accuracy
np.random.seed(42)
x_train += np.random.normal(0, 1.5, x_train.shape)  # Stronger noise
x_test += np.random.normal(0, 1.5, x_test.shape)

# Randomly shuffle 30% of the labels to introduce misclassification
num_labels_to_shuffle = int(0.3 * len(y_train))
shuffle_indices = np.random.choice(len(y_train), num_labels_to_shuffle, replace=False)
y_train[shuffle_indices] = 1 - y_train[shuffle_indices]

# Remove one feature to reduce model effectiveness
x_train = np.delete(x_train, 1, axis=1)  # Removing "battery_current"
x_test = np.delete(x_test, 1, axis=1)

# Reduce training data size
subset_indices = np.random.choice(len(x_train), size=int(0.5 * len(x_train)), replace=False)
x_train = x_train[subset_indices]
y_train = y_train[subset_indices]

# Train an XGBoost classifier with altered hyperparameters
xgb_model = XGBClassifier(n_estimators=2, max_depth=1, learning_rate=0.7)  # Fewer estimators, higher learning rate
xgb_model.fit(x_train, y_train)

# Test the model
predictions = xgb_model.predict(x_test)

# Calculate metrics
accuracy = accuracy_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

# Print results
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R2 Score: {r2:.4f}")


Test Accuracy: 100.00%
Mean Absolute Error (MAE): 0.0000
Mean Squared Error (MSE): 0.0000
Root Mean Squared Error (RMSE): 0.0037
R2 Score: -0.0000


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

# Load datasets
broken_bag_path = "broken_bag.csv"
healthy_bag_path = "healthy_bag.csv"

broken_bag_data = pd.read_csv(broken_bag_path)
healthy_bag_data = pd.read_csv(healthy_bag_path)

# Select features
features = ["battery_voltage", "battery_current", "battery_energy"]

x_train = healthy_bag_data[features].values
y_train = (healthy_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values  # Binary classification

x_test = broken_bag_data[features].values
y_test = (broken_bag_data["scaled_adjusted_battery_energy"] < 0).astype(int).values

# Normalize data
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Add strong Gaussian noise to reduce accuracy
noise_factor = 0.5  # Increased noise
x_train += noise_factor * np.random.normal(size=x_train.shape)
x_test += noise_factor * np.random.normal(size=x_test.shape)

# Reshape for CNN input
x_train = x_train.reshape(-1, len(features), 1)
x_test = x_test.reshape(-1, len(features), 1)

# Define a minimal CNN model
def create_cnn_model(input_shape):
    model = models.Sequential([
        layers.Conv1D(4, 2, activation='relu', padding='same', input_shape=input_shape),  # Reduced filters
        layers.Dropout(0.4),  # Higher dropout
        layers.Flatten(),
        layers.Dropout(0.5),  # Increased dropout
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize model
input_shape = x_train.shape[1:]
model = create_cnn_model(input_shape)

# Add Early Stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss', patience=2, restore_best_weights=True
)

# Train the model
history = model.fit(
    x_train, y_train,
    epochs=10, batch_size=32,
    validation_split=0.3,
    callbacks=[early_stopping]
)

# Test the model
predictions = model.predict(x_test)
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Test Accuracy on broken_bag data: {accuracy * 100:.2f}%")

# Additional metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
eps = 1e-9  # To avoid log(0)
log_odds = np.log(predictions / (1 - predictions + eps))
r2 = r2_score(y_test, log_odds)

#r2 = max(0, r2_score(y_test, predictions))

#r2 = r2_score(y_test, predicted_labels)  # Updated R² calculation

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4835/4835 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8336 - loss: 0.4026 - val_accuracy: 1.0000 - val_loss: 0.0080
Epoch 2/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9138 - loss: 0.2044 - val_accuracy: 1.0000 - val_loss: 0.0037
Epoch 3/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9264 - loss: 0.1824 - val_accuracy: 1.0000 - val_loss: 0.0029
Epoch 4/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9279 - loss: 0.1770 - val_accuracy: 1.0000 - val_loss: 0.0029
Epoch 5/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9285 - loss: 0.1749 - val_accuracy: 1.0000 - val_loss: 0.0028
Epoch 6/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9285 - loss: 0.1748 - val_accuracy: 1.0000 - val_loss: 0.0027
Epoch 7/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9285 - loss: 0.1744 - val_accuracy: 1.0000 - val_loss: 0.0032
Epoch 8/10
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9267 - loss: 0.1775 - val_ac